In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thymoma"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Thymoma"
in_cohort_dir = "../../input/GEO/Thymoma/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Thymoma/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Thymoma/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Thymoma/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Thymoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE131027_family.soft.gz', 'GSE131027_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Thymoma/GSE131027/GSE131027_family.soft.gz
Matrix file: ../../input/GEO/Thymoma/GSE131027/GSE131027_series_matrix.txt.gz


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to contain genetic mutation data rather than gene expression data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For Thymoma, we need to check the cancer type list in row 1
trait_row = 1  # cancer type data is in row 1

# Age is not available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(x):
    """Convert trait data to binary format (1 for Thymoma, 0 for others)"""
    if pd.isna(x) or x is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if isinstance(x, str) and ':' in x:
        x = x.split(':', 1)[1].strip()
    
    # Check if it's Thymoma (case insensitive)
    if x.lower() == 'thymoma':
        return 1
    else:
        return 0

# Define these functions even though we don't have the data
def convert_age(x):
    """Convert age data to continuous format"""
    if pd.isna(x) or x is None:
        return None
    
    if isinstance(x, str) and ':' in x:
        value = x.split(':', 1)[1].strip()
        try:
            return float(value)
        except ValueError:
            return None
    return None

def convert_gender(x):
    """Convert gender data to binary format (0 for female, 1 for male)"""
    if pd.isna(x) or x is None:
        return None
    
    if isinstance(x, str) and ':' in x:
        value = x.split(':', 1)[1].strip().lower()
        if 'female' in value or 'f' == value:
            return 0
        elif 'male' in value or 'm' == value:
            return 1
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # This dictionary was provided in the "Output of a previous step"
    sample_chars_dict = {
        0: ['tissue: tumor biopsy'], 
        1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 
            'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 
            'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 
            'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 
            'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 
            'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 
            'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 'cancer: Neuroendocrine cancer', 
            'cancer: vulvovaginal'], 
        2: ['mutated gene: ATR', 'mutated gene: FAN1', 'mutated gene: ERCC3', 'mutated gene: FANCD2', 
            'mutated gene: BAP1', 'mutated gene: DDB2', 'mutated gene: TP53', 'mutated gene: ATM', 
            'mutated gene: CHEK1', 'mutated gene: BRCA1', 'mutated gene: WRN', 'mutated gene: CHEK2', 
            'mutated gene: BRCA2', 'mutated gene: XPC', 'mutated gene: PALB2', 'mutated gene: ABRAXAS1', 
            'mutated gene: NBN', 'mutated gene: BLM', 'mutated gene: FAM111B', 'mutated gene: FANCA', 
            'mutated gene: MLH1', 'mutated gene: BRIP1', 'mutated gene: IPMK', 'mutated gene: RECQL', 
            'mutated gene: RAD50', 'mutated gene: FANCM', 'mutated gene: GALNT12', 'mutated gene: SMAD9', 
            'mutated gene: ERCC2', 'mutated gene: FANCC'], 
        3: ['predicted: HRDEXP: HRD', 'predicted: HRDEXP: NO_HRD'], 
        4: ['parp predicted: kmeans-2: PARP sensitive', 'parp predicted: kmeans-2: PARP insensitive']
    }
    
    # Convert to DataFrame format
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


A new JSON file was created at: ../../output/preprocess/Thymoma/cohort_info.json
Clinical Data Preview:
{0: [0.0], 1: [0.0], 2: [0.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [0.0], 7: [0.0], 8: [0.0], 9: [0.0], 10: [0.0], 11: [0.0], 12: [0.0], 13: [0.0], 14: [0.0], 15: [0.0], 16: [1.0], 17: [0.0], 18: [0.0], 19: [0.0], 20: [0.0], 21: [0.0], 22: [0.0], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Thymoma/clinical_data/GSE131027.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the previous step
# The format "1007_s_at", "1053_at", etc. indicates these are Affymetrix probe IDs,
# not standard HGNC gene symbols
# These probe IDs need to be mapped to gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation data correspond to 
# gene identifiers and gene symbols
# Looking at the preview, we can see:
# - 'ID' column contains probe IDs like "1007_s_at" which match the gene expression data
# - 'Gene Symbol' column contains the human gene symbols we need to map to

# 2. Get gene mapping dataframe with the identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print some information about the mapping
print("Gene mapping dataframe shape:", gene_mapping.shape)
print("Sample of gene mapping dataframe:")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# First, get the gene expression data from the matrix file
genetic_data = get_genetic_data(matrix_file)

# Then apply the mapping to convert probe IDs to gene symbols
gene_data = apply_gene_mapping(expression_df=genetic_data, mapping_df=gene_mapping)

# Print information about the generated gene expression data
print("Gene expression dataframe shape:", gene_data.shape)
print("First few gene symbols in gene expression data:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Sample of gene mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Gene expression dataframe shape: (21278, 92)
First few gene symbols in gene expression data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Thymoma/gene_data/GSE131027.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
print(f"Sample IDs in gene data: {gene_data.columns[:5].tolist()}...")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Reload clinical data and properly process it for this Thymoma dataset
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Define conversion functions specifically for the Thymoma dataset
def convert_trait(value):
    """Convert to binary: 0 for cell lines, 1 for tumor samples"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Cell lines are encoded as "CL"
    if value == "CL":
        return 0
    # All other types are tumor samples
    else:
        return 1

# Extract the clinical data using the appropriate rows based on our analysis
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=1,  # Row for tumor type (from the sample characteristics dictionary)
    convert_trait=convert_trait,
    # No age or gender data available in this dataset
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Handle sample ID format to ensure proper linking
# Strip quotes from sample IDs if present
gene_data.columns = gene_data.columns.str.strip('"')
clinical_df.columns = clinical_df.columns.str.strip('"')

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from thymic tumors. The trait variable represents tumor type (0=Cell line, 1=Thymic tumor)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Shape of gene data after normalization: (19845, 92)
First few gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']
Sample IDs in gene data: ['GSM3759992', 'GSM3759993', 'GSM3759994', 'GSM3759995', 'GSM3759996']...


Normalized gene data saved to ../../output/preprocess/Thymoma/gene_data/GSE131027.csv


Clinical data preview:
{'GSM3759992': [1.0], 'GSM3759993': [1.0], 'GSM3759994': [1.0], 'GSM3759995': [1.0], 'GSM3759996': [1.0], 'GSM3759997': [1.0], 'GSM3759998': [1.0], 'GSM3759999': [1.0], 'GSM3760000': [1.0], 'GSM3760001': [1.0], 'GSM3760002': [1.0], 'GSM3760003': [1.0], 'GSM3760004': [1.0], 'GSM3760005': [1.0], 'GSM3760006': [1.0], 'GSM3760007': [1.0], 'GSM3760008': [1.0], 'GSM3760009': [1.0], 'GSM3760010': [1.0], 'GSM3760011': [1.0], 'GSM3760012': [1.0], 'GSM3760013': [1.0], 'GSM3760014': [1.0], 'GSM3760015': [1.0], 'GSM3760016': [1.0], 'GSM3760017': [1.0], 'GSM3760018': [1.0], 'GSM3760019': [1.0], 'GSM3760020': [1.0], 'GSM3760021': [1.0], 'GSM3760022': [1.0], 'GSM3760023': [1.0], 'GSM3760024': [1.0], 'GSM3760025': [1.0], 'GSM3760026': [1.0], 'GSM3760027': [1.0], 'GSM3760028': [1.0], 'GSM3760029': [1.0], 'GSM3760030': [1.0], 'GSM3760031': [1.0], 'GSM3760032': [1.0], 'GSM3760033': [1.0], 'GSM3760034': [1.0], 'GSM3760035': [1.0], 'GSM3760036': [1.0], 'GSM3760037': [1.0], 'GSM376003

Shape of linked data after handling missing values: (92, 19846)
Quartiles for 'Thymoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Thymoma' in this dataset is severely biased.

Dataset validation failed. Final linked data not saved.
